In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

In [2]:
class Net(nn.Module):
    def init(self):
        super().init()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
class CNN(nn.Module):
    def __init__(self,activation_function):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 6, 3),#32x32 -> 30x30 (no pad)
            activation_function(),
            nn.MaxPool2d(2, stride = 2), #2x2 maxpool (devide hight and with by 2 -> 15x15)
            nn.Conv2d(6, 10, 3),#(no padding -> 13x13)
            activation_function(),
        )
        self.linear_lay = nn.Sequential(
            nn.Linear(1690, 300),# (13X13X10 (outchanels))
            activation_function(),
            nn.Linear(300, 50),
            activation_function(),
            nn.Linear(50,10)
        )
        
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0),-1)# this flattens X before linear layer
        x = self.linear_lay(x) # bc this is our output layer. No activation here.
        
        return F.softmax(x, dim=1)
            
        

In [4]:
# Writer will output to ./runs/ directory by default
writer = SummaryWriter()

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
X_train, X_val= train_test_split(
trainset, test_size=0.2, random_state=42)

trainloader = torch.utils.data.DataLoader(X_train, batch_size=4,
                                          shuffle=True, num_workers=2)
validationloader = torch.utils.data.DataLoader(X_val, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [5]:
import matplotlib.pyplot as plt

def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
import torch
train_on_gpu = torch.cuda.is_available()
print(train_on_gpu)
device = torch.device("cuda:0")
device

True


device(type='cuda', index=0)

In [6]:
print(len(validationloader))
print(len(trainloader))

2500
10000


In [10]:
import torch.optim as optim
PATH='model.pth'
net = CNN(nn.LeakyReLU)
net = net.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        writer.add_scalar('Model1 Loss/train', loss.item(), epoch*10000+i) 
        running_loss += loss.item()
        
        if i % 2000 == 1999:    # print every 2000 mini-batches
            val_loss = validate(net, 1, epoch)
            print('[%d, %5d] loss: %.3f %d porcent correct ' %
                  (epoch + 1, i + 1, running_loss / 2000, val_loss))
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 2.302 10 porcent correct 
[1,  4000] loss: 2.303 10 porcent correct 
[1,  6000] loss: 2.302 11 porcent correct 
[1,  8000] loss: 2.303 11 porcent correct 
[1, 10000] loss: 2.302 12 porcent correct 
[2,  2000] loss: 2.302 12 porcent correct 
[2,  4000] loss: 2.302 12 porcent correct 
[2,  6000] loss: 2.302 13 porcent correct 
[2,  8000] loss: 2.302 13 porcent correct 
[2, 10000] loss: 2.302 12 porcent correct 
[3,  2000] loss: 2.302 14 porcent correct 
[3,  4000] loss: 2.302 14 porcent correct 
[3,  6000] loss: 2.302 14 porcent correct 
[3,  8000] loss: 2.301 13 porcent correct 
[3, 10000] loss: 2.301 13 porcent correct 
[4,  2000] loss: 2.301 14 porcent correct 
[4,  4000] loss: 2.301 13 porcent correct 
[4,  6000] loss: 2.301 14 porcent correct 
[4,  8000] loss: 2.300 13 porcent correct 
[4, 10000] loss: 2.300 13 porcent correct 
[5,  2000] loss: 2.299 13 porcent correct 
[5,  4000] loss: 2.298 14 porcent correct 
[5,  6000] loss: 2.297 15 porcent correct 
[5,  8000] 

In [12]:
net2 = CNN(nn.LeakyReLU)
net2 = net2.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net2.parameters(), lr=0.0001)
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        writer.add_scalar('Model2 Loss/train', loss.item(), epoch*10000+i) 
        
        if i % 2000 == 1999:    # print every 2000 mini-batches
            val_loss = validate(net2, 2, epoch)
            print('[%d, %5d] loss: %.3f %d procent correct ' %
                  (epoch + 1, i + 1, running_loss / 2000, val_loss))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.203 27 procent correct 
[1,  4000] loss: 2.139 32 procent correct 
[1,  6000] loss: 2.110 34 procent correct 
[1,  8000] loss: 2.100 35 procent correct 
[1, 10000] loss: 2.089 37 procent correct 
[2,  2000] loss: 2.065 40 procent correct 
[2,  4000] loss: 2.057 39 procent correct 
[2,  6000] loss: 2.051 40 procent correct 
[2,  8000] loss: 2.043 39 procent correct 
[2, 10000] loss: 2.041 42 procent correct 
[3,  2000] loss: 2.032 44 procent correct 
[3,  4000] loss: 2.017 42 procent correct 
[3,  6000] loss: 2.019 42 procent correct 
[3,  8000] loss: 2.015 43 procent correct 
[3, 10000] loss: 2.001 42 procent correct 
[4,  2000] loss: 2.000 43 procent correct 
[4,  4000] loss: 1.988 44 procent correct 
[4,  6000] loss: 1.994 47 procent correct 
[4,  8000] loss: 1.990 43 procent correct 
[4, 10000] loss: 1.989 45 procent correct 
[5,  2000] loss: 1.979 48 procent correct 
[5,  4000] loss: 1.974 47 procent correct 
[5,  6000] loss: 1.971 49 procent correct 
[5,  8000] 

In [13]:
net3 = CNN(nn.Tanh)
net3 = net3.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net3.parameters(), lr=0.0001)
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net3(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        writer.add_scalar('Model3 Loss/train', loss.item(), epoch*10000+i) 
        
        if i % 2000 == 1999:    # print every 2000 mini-batches
            val_loss = validate(net3, 3, epoch)
            print('[%d, %5d] loss: %.3f %d procent correct ' %
                  (epoch + 1, i + 1, running_loss / 2000, val_loss))
            running_loss = 0.0


print('Finished Training')

[1,  2000] loss: 2.206 31 procent correct 
[1,  4000] loss: 2.148 33 procent correct 
[1,  6000] loss: 2.113 34 procent correct 
[1,  8000] loss: 2.094 35 procent correct 
[1, 10000] loss: 2.086 39 procent correct 
[2,  2000] loss: 2.062 40 procent correct 
[2,  4000] loss: 2.044 42 procent correct 
[2,  6000] loss: 2.032 44 procent correct 
[2,  8000] loss: 2.029 44 procent correct 
[2, 10000] loss: 2.014 43 procent correct 
[3,  2000] loss: 2.003 46 procent correct 
[3,  4000] loss: 1.997 46 procent correct 
[3,  6000] loss: 1.986 46 procent correct 
[3,  8000] loss: 1.988 47 procent correct 
[3, 10000] loss: 1.981 49 procent correct 
[4,  2000] loss: 1.968 49 procent correct 
[4,  4000] loss: 1.963 47 procent correct 
[4,  6000] loss: 1.963 50 procent correct 
[4,  8000] loss: 1.952 49 procent correct 
[4, 10000] loss: 1.948 50 procent correct 
[5,  2000] loss: 1.934 50 procent correct 
[5,  4000] loss: 1.943 50 procent correct 
[5,  6000] loss: 1.935 51 procent correct 
[5,  8000] 

In [9]:
def test(model, modelVal):
    true_labels=[]
    pred_labels=[]
    correct = 0
    total = 0
    losses = []
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            image, label = data
            image, label = image.to(device), label.to(device)
            pred = model(image)
            for i in range(4):
                if torch.argmax(pred[i]) == label[i]:
                    correct += 1
                total += 1
    print(" predicted = ", correct/total*100, "% of total ",total)
    writer.add_scalar('Model' + str(modelVal) +'/Accuracy/test', correct/total*100)

In [8]:
def validate(model, modelVal, epoch):
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            image, label = data
            image, label = image.to(device), label.to(device)
            pred = model(image)
            outputs = model(image)
            loss = criterion(outputs, label)
                         
            if torch.argmax(pred[0]) == label[0]:
                correct += 1
            total += 1
        writer.add_scalar('Model' + str(modelVal) + '/Epoch' + str(epoch) + '/Loss/validation', loss.item())
    return(correct/total*100)

In [16]:
test(net, 1)

 predicted =  35.08 % of total  10000


In [17]:
test(net2, 2)

 predicted =  58.650000000000006 % of total  10000


In [18]:
test(net3, 3)

 predicted =  58.3 % of total  10000
